In [1]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')

In [2]:
ls

 Volume in drive C is Windows
 Volume Serial Number is 48D9-C9EA

 Directory of C:\Users\hli378\Documents\rec sys\movie recommendation

11/05/2018  12:55 PM    <DIR>          .
11/05/2018  12:55 PM    <DIR>          ..
11/05/2018  12:55 PM    <DIR>          .ipynb_checkpoints
10/09/2018  09:41 AM       189,917,659 credits.csv
10/09/2018  09:41 AM         6,231,943 keywords.csv
10/09/2018  09:41 AM           989,107 links.csv
10/09/2018  09:41 AM           183,372 links_small.csv
11/01/2018  03:06 PM            75,181 movie_recommender.ipynb
11/05/2018  12:52 PM            21,801 movie_recommender_new.ipynb
10/09/2018  09:41 AM        34,445,126 movies_metadata.csv
10/09/2018  09:41 AM       709,550,327 ratings.csv
10/09/2018  09:41 AM         2,438,266 ratings_small.csv
10/17/2018  12:56 PM             7,485 Untitled.ipynb
              10 File(s)    943,860,267 bytes
               3 Dir(s)  664,516,542,464 bytes free


In [4]:
keywords=pd.read_csv('keywords.csv')
keywords.head(4)

,id,keywords
0,862,"[{'id': 931, 'name': 'jealousy'}, {'id': 4290,..."
1,8844,"[{'id': 10090, 'name': 'board game'}, {'id': 1..."
2,15602,"[{'id': 1495, 'name': 'fishing'}, {'id': 12392..."
3,31357,"[{'id': 818, 'name': 'based on novel'}, {'id':..."


In [3]:
links=pd.read_csv("links_small.csv")
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [2]:
movies=pd.read_csv('movies_metadata.csv')
movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
movies.iloc[0]

adult                                                                False
belongs_to_collection    {'id': 10194, 'name': 'Toy Story Collection', ...
budget                                                            30000000
genres                   [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
homepage                              http://toystory.disney.com/toy-story
id                                                                     862
imdb_id                                                          tt0114709
original_language                                                       en
original_title                                                   Toy Story
overview                 Led by Woody, Andy's toys live happily in his ...
popularity                                                         21.9469
poster_path                               /rhIRbceoE9lR4veEXuwCC2wARtG.jpg
production_companies        [{'name': 'Pixar Animation Studios', 'id': 3}]
production_countries     

In [9]:
movies.shape

(45466, 24)

# find similar items

## this may have other solid methods to fill in 

In [4]:
len(movies[movies['overview'].isnull()])

954

In [5]:
movies['overview']=movies['overview'].fillna('')

In [6]:
len(movies[movies['overview'].isnull()])

0

In [38]:
movies_small=movies.sample(frac=0.3).copy()

In [39]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
import time
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies_small['overview'])

In [40]:
t=time.time()
print('generate cosine similarity')
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print('time used',time.time()-t)

generate cosine similarity
time used 1.6725471019744873


# find most similar 10

In [41]:
cosine_sim.shape

(13640, 13640)

In [57]:
movie_dictionary={}
ind=0
for i,row in movies_small.iterrows():
    movie_dictionary[ind]=row['id']
    ind+=1

In [64]:
similars=[]
for i in list(np.argsort(cosine_sim,axis=1)[:,-11:]):
    similars.append([movie_dictionary.get(j) for j in i][::-1])
movie_similar=pd.DataFrame({'id':movies_small['id'],'similar':similars})

# recommender

In [66]:
for ind, row in movie_similar.head(10).iterrows():
    print('-----------------')
    print(movies_small.loc[movies_small['id']==row['id'],'title'])
    print('similar to')
    print(movies_small.loc[movies_small['id'].isin(row['similar']),'title'])

-----------------
39433    Lifeline
Name: title, dtype: object
similar to
39433                                 Lifeline
41272                            Outlaw: Kill!
43352                               Bloodfight
14218                                      Ink
14778                         Edge of Darkness
30512                              Best in Bed
4394                                Masquerade
17398                            A Perfect Day
44884    Site Unseen: An Emma Fielding Mystery
814                                       Emma
16723                                     Emma
Name: title, dtype: object
-----------------
36783    The Steel Road
Name: title, dtype: object
similar to
36783                               The Steel Road
27771                                     The Shoe
15770                              Arch of Triumph
20301    Enola Gay and the Atomic Bombing of Japan
12285               Walk Hard: The Dewey Cox Story
15081                                   Wild Ch

# collaborative filtering

In [67]:
#%% collaborative filtering
from surprise import Reader, Dataset, SVD, KNNBasic, evaluate,accuracy
from surprise.model_selection import cross_validate
from collections import defaultdict
import time

In [68]:
ratings=pd.read_csv('ratings_small.csv')
ratings.head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [71]:
import random
raw_ratings = data.raw_ratings

# shuffle ratings 
random.shuffle(raw_ratings)

# build training(90%) and testing (10%)
# A = 90% of the data, B = 10% of the data
threshold = int(.9 * len(raw_ratings))
training_ratings = raw_ratings[:threshold]
testing_ratings = raw_ratings[threshold:]

In [72]:
# training set, cross-validation to tune parameters
data.raw_ratings = training_ratings  

# grid search to find best score plus cross-validation
print('Grid Search...')
import time
t=time.time()
from surprise.model_selection import GridSearchCV
param_grid = {'n_factors':[10],'n_epochs': [5, 10], 'lr_all': [0.002], 'reg_all': [0.4],'verbose':[True]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

# best RMSE score
print(gs.best_score['rmse'])
# combination of parameters that gave the best RMSE score
print(gs.best_params['rmse'])

results_df_svd = pd.DataFrame.from_dict(gs.cv_results)
#print(results_df_svd)

# use the best estimator as the final model
model_final_svd = gs.best_estimator['rmse']
print('time used',time.time()-t)

Grid Search...
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
0.9329830429554002
{'n_factors': 10, 'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.4, 'verbose': True}
time used 6.356888294219971

In [73]:
# retrain on the training set 
trainset = data.build_full_trainset()
model_final_svd.fit(trainset)

# Compute biased accuracy on training
predictions = model_final_svd.test(trainset.build_testset())
print('Biased accuracy on training set,', end='   ')
accuracy.rmse(predictions)

# Compute unbiased accuracy on testing
testset = data.construct_testset(testing_ratings)  # testset is now the set B
predictions = model_final_svd.test(testset)
print('Unbiased accuracy on testing set,', end=' ')
accuracy.rmse(predictions)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Biased accuracy on training set,   RMSE: 0.9069
Unbiased accuracy on testing set, RMSE: 0.9192


0.9191553984936599

In [74]:
def precision_recall_at_k(predictions, k=10, threshold=3.5):
    '''Return precision and recall at k metrics for each user.'''

    # First map the predictions to each user.
    user_est_true = defaultdict(list)
    for uid, _, true_r, est, _ in predictions:
        user_est_true[uid].append((est, true_r))

    precisions = dict()
    recalls = dict()
    for uid, user_ratings in user_est_true.items():

        # Sort user ratings by estimated value
        user_ratings.sort(key=lambda x: x[0], reverse=True)

        # Number of relevant items
        n_rel = sum((true_r >= threshold) for (_, true_r) in user_ratings)

        # Number of recommended items in top k
        n_rec_k = sum((est >= threshold) for (est, _) in user_ratings[:k])

        # Number of relevant and recommended items in top k
        n_rel_and_rec_k = sum(((true_r >= threshold) and (est >= threshold))
                              for (est, true_r) in user_ratings[:k])

        # Precision@K: Proportion of recommended items that are relevant
        precisions[uid] = n_rel_and_rec_k / n_rec_k if n_rec_k != 0 else 1

        # Recall@K: Proportion of relevant items that are recommended
        recalls[uid] = n_rel_and_rec_k / n_rel if n_rel != 0 else 1

    return precisions, recalls

In [76]:
evaluation_results=precision_recall_at_k(predictions, k=10, threshold=3.5)